In [6]:
#importing required packages
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import random
import time
from time import sleep
import csv

In [14]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import re
nlp = spacy.load('en_core_web_sm')
import joblib

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

In [16]:
tf_idf = joblib.load('tf_idf.pkl')
scaler = joblib.load('scaler.pkl')
clf = joblib.load('clf.pkl')


In [17]:
def waitTime(min,max):
	'''
	Waits for random seconds between min and max.
	'''
	secs = random.uniform(min,max)
	sleep(secs)
	

In [11]:
#Path of chromedriver
path = 'C:\\Program Files (x86)\\chromedriver.exe'
driver = webdriver.Chrome(path)
#driver = webdriver.Chrome()
driver.get('https://www.okcupid.com/discover')
#Login manually and verify using otp

C:\Users\RAMANA~1\AppData\Local\Temp/ipykernel_4476/2517153681.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [18]:
#Defines Empty List array
profiledata= [];

In [28]:
def scrolldown():

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        waitTime(1,2)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height-250:
            break
        last_height = new_height+250

In [61]:
def swipe_profiles(num):
    
    
    def clickbutton(button):

        element = driver.find_element(By.CLASS_NAME,button)

        actions = ActionChains(driver)
        actions.move_to_element(element).perform()

        #Click pass button
        wait = WebDriverWait(driver, 20)
        element = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "dt-action-buttons-button.pass")))
        element.click()

        
    for i in range(num):
        profiledata= [];
        driver.execute_script("window.scrollTo(0, 500);")
        print(f'Accessing profile #{i+1}')

        waitTime(1.5,2.5)
        pass_button = driver.find_elements(By.CLASS_NAME,'dt-action-buttons-button')[0]
        
        like_button = driver.find_elements(By.CLASS_NAME,'dt-action-buttons-button')[1]
        
        superlike_button = driver.find_element(By.CLASS_NAME,'superlike-button-scene.superlike-button-scene--pill')
        
        top = driver.find_element(By.CLASS_NAME,'desktop-dt-top')

        
        
        try:
            bio = driver.find_element(By.CLASS_NAME,'dt-essay-text').text
        except:
            bio = ''
            print('Error during Bio')

        try:
            ageLoc = driver.find_element(By.CLASS_NAME,'card-content-header__location').text
        except:
            ageLoc = ''
            print('Error during AgeLoc')
        try:
            
            scrolldown()
            questions = driver.find_elements(By.CLASS_NAME,'profile-questions-filter-count')
        except:
            questions = ''
            print('Error during Questions')
        name = driver.find_element(By.CLASS_NAME,'card-content-header__text').text
        ageLoc = driver.find_element(By.CLASS_NAME,'card-content-header__location').text
        questions = driver.find_elements(By.CLASS_NAME,'profile-questions-filter-count')
        #(like,disagree,findout are elements of questions) 

        #Appends all data to the profiledata list
        profiledata.append([ageLoc,bio,questions[0].text,questions[1].text,questions[2].text])
        print([ageLoc,bio,questions[0].text,questions[1].text,questions[2].text]);
        prediction = get_pred(profiledata)
        
        if prediction == 1:
            print("Profile is fake/low info. Swiping Left 🐱 ")
            #Code to click pass
            button = 'dt-action-buttons-button.pass'
            clickbutton(button)
        elif prediction == 0:
            #Code to click like
            print("Profile is Ok. Swiping Right 🐶")
            button = 'dt-action-buttons-button.like'
            clickbutton(button)
            




In [62]:
## Function to pre-process string. Replace emojis and numbers with strings.
def prepro_string(bio):
    bio = str(bio)
    bio = remove_emoji(bio.lower()) ## Parse Emojis
    new_bio = '';
    bio = bio.replace("\n","")  #Removes newline characters
    doc = nlp(bio)
    
    for tok in doc:             ##Convert Digits into Digit(Num_digits) Format.

        if tok.is_digit:
            digit_len = len(tok)
            new_bio += str(digit_len) + '_Digit_' + 'Num'  + ' '
        else:
            new_bio += tok.text + ' '

    return new_bio

In [63]:
## Function to remove emojis and replace them with the string-'emoji' & remove punctuation
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" # emoticons
                           u"\U0001F300-\U0001F5FF" # symbols & pictographs
                           u"\U0001F680-\U0001F6FF" # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]", flags=re.UNICODE)
    
    string = re.sub(r'[^\w\s]', '', string)


    return emoji_pattern.sub(' emoji ', string)

In [64]:
##Function to count length of bio
def get_char_counts(string):
    #initialized value
    total_digits = 0
    total_letters = 0

    # iterate through all characters
    for s in string:

        # if character is digit (return True)
        if s.isnumeric():
            total_digits += 1

        # if character is letter (return False)
        else:
            total_letters += 1
    return total_digits, total_letters

In [65]:
def get_pred(profiledata):
    for i in range(2,5):
        profiledata[0][i] = int(profiledata[0][i])
    df = pd.DataFrame(profiledata,columns =['Age . Location','Bio','Agree','Disagree','Find Out'])
    
    ## Getting number of characters and number of digits in the Bio.
    applied_df = df.apply(lambda row: get_char_counts(row.Bio), axis='columns', result_type='expand')
    df = pd.concat([df, applied_df], axis='columns')
    df = df.rename(columns={0:'Num_Digits',1:'Num_Chars'})
    
    #Applying Preprocessing Function to Bio
    df['Bio'] = df['Bio'].apply(prepro_string)
    
    df['Ques_Ans'] = df['Agree'] + df['Disagree'] + df['Find Out']
    
    X = df.drop(['Age . Location','Agree','Disagree','Find Out','Num_Digits'],axis=1)
    scaled_X_test_nums = scaler.transform(X[['Ques_Ans','Num_Chars']])
    X_test_gram = tf_idf.transform(X['Bio'])
    #Test Data
    test_arr = np.array(scaled_X_test_nums)
    M_test = sparse.coo_matrix(test_arr)
    sparse_X_test = sparse.hstack((M_test, X_test_gram))
    
    y_pred = clf.predict(sparse_X_test)
    
    return y_pred[-1]

## Model Deployment

In [68]:
swipe_profiles(5)

Accessing profile #1
['23 • Husain Sawāli Dargāh', 'love meeting new friends', '9', '6', '0']
Profile is Ok. Swiping Right 🐶
Accessing profile #2
['24 • Delhi', 'Looking for something serious\n\nNo matter how do u look , your nature plays a pivotal role in my life..\n\nLooking for someone serious, intellectual, decent ,and must have good dressing sense\n\nI am very much kiddish so beware of me\nI can tease you anytime as I love to tease people especially the boring people.', '14', '14', '3']
Profile is Ok. Swiping Right 🐶
Accessing profile #3
['27 • Kūkatpalli', 'hi', '9', '6', '0']
Profile is fake/low info. Swiping Left 🐱 
Accessing profile #4
['25 • Bengaluru', 'Most people that know me would say\nI’m most likely to find reasons to order takeout, least likely to cook', '39', '8', '135']
Profile is Ok. Swiping Right 🐶
Accessing profile #5
['21 • Hyderābād', 'Heyyy !\nI am 21 year old business student ! Looking for good company & friendship !', '6', '9', '0']
Profile is Ok. Swiping Rig